

Tôi sẽ sửa lại `def_promptB.py` để có thể linh hoạt chuyển đổi giữa OpenAI và API mới:

```python:6_TuningWith2Prompting/def_promptB.py
import time
import json
import requests
from openai import OpenAI

class AICoachAPI:
    def __init__(self, base_url="http://103.253.20.13:9400"):
        self.base_url = base_url
        self.init_endpoint = f"{base_url}/personalized-ai-coach/api/v1/bot/initConversation"
        self.webhook_endpoint = f"{base_url}/personalized-ai-coach/api/v1/bot/webhook"

    def init_conversation(self, bot_id=3, conversation_id=None):
        if conversation_id is None:
            conversation_id = f"conv_{int(time.time())}"
        
        payload = {
            "bot_id": bot_id,
            "conversation_id": conversation_id,
            "input_slots": {}
        }
        
        try:
            response = requests.post(
                self.init_endpoint,
                headers={'Content-Type': 'application/json'},
                json=payload
            )
            response.raise_for_status()
            return response.json(), conversation_id
        except Exception as e:
            print(f"Error initializing conversation: {str(e)}")
            return None, None

    def send_message(self, conversation_id, message):
        payload = {
            "conversation_id": conversation_id,
            "message": message
        }
        
        try:
            response = requests.post(
                self.webhook_endpoint,
                headers={'Content-Type': 'application/json'},
                json=payload
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error sending message: {str(e)}")
            return None

def generate_roleB_response(client, roleB_prompt, message_history, use_api=False):
    """
    Generate response for roleB
    Args:
        client: OpenAI client or AICoachAPI instance
        roleB_prompt: Prompt for roleB
        message_history: Conversation history
        use_api: If True, use AICoachAPI, if False use OpenAI
    """
    print("\n=== RoleB Turn ===")
    print("Original message history:")
    print(json.dumps(message_history, indent=2, ensure_ascii=False))
    
    start_time = time.time()
    
    if use_api:
        # Using AI Coach API
        if not hasattr(generate_roleB_response, 'conversation_id'):
            # Initialize conversation if not already done
            _, conversation_id = client.init_conversation()
            if conversation_id:
                generate_roleB_response.conversation_id = conversation_id
            else:
                raise Exception("Failed to initialize conversation")
        
        # Get last message or use default
        last_message = message_history[-1]['content'] if message_history else "sẵn sàng"
        
        # Send message
        response = client.send_message(
            conversation_id=generate_roleB_response.conversation_id,
            message=last_message
        )
        
        if not response:
            raise Exception("Failed to get response from API")
        
        roleB_message = response.get('message', '')
        
    else:
        # Using OpenAI
        api_messages = [{"role": "system", "content": roleB_prompt}]
        if message_history:
            converted_history = convert_roles_for_api(message_history, is_roleA_turn=False)
            api_messages.extend(converted_history)
            print("\nConverted history for RoleB:")
            print(json.dumps(api_messages, indent=2, ensure_ascii=False))
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=api_messages,
            temperature=0
        )
        roleB_message = response.choices[0].message.content
    
    end_time = time.time()
    print(f"\nRoleB Response: {roleB_message}")
    print(f"Response Time: {end_time - start_time:.2f}s")
    
    message_history.append({"role": "roleB", "content": roleB_message})
    return message_history, end_time - start_time

# Helper function to create appropriate client
def create_client(use_api=False):
    """Create either OpenAI client or AICoachAPI client"""
    if use_api:
        return AICoachAPI()
    else:
        return OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
```

Cách sử dụng trong `main_v2.py`:

```python:6_TuningWith2Prompting/main_v2.py
from def_promptB import generate_roleB_response, create_client

# Tạo client tùy theo nhu cầu
use_api = True  # hoặc False để dùng OpenAI
client = create_client(use_api)

# Trong hàm simulate_conversation
def simulate_conversation(row):
    # ... code khác giữ nguyên ...
    
    while conversationTurnCount < maxTurns:
        try:
            # RoleA's turn (giữ nguyên)
            message_history, roleA_time = generate_roleA_response(openai_client, roleA_prompt, message_history)
            response_times.append(roleA_time)

            # RoleB's turn (thêm use_api parameter)
            message_history, roleB_time = generate_roleB_response(
                client, 
                roleB_prompt, 
                message_history,
                use_api=use_api
            )
            response_times.append(roleB_time)

            conversationTurnCount += 1
```

Những điểm chính:
1. Thêm parameter `use_api` để chọn phương thức
2. Tách AICoachAPI thành class riêng
3. Lưu conversation_id dùng static variable
4. Hàm helper `create_client()` để tạo client phù hợp
5. Giữ nguyên format input/output để tương thích với code cũ

Bạn có thể dễ dàng chuyển đổi giữa 2 phương thức bằng cách thay đổi `use_api=True/False`.
